# 设计自定义层 
我看先来看如何定义一个简单层，它不需要维护模型参数。事实上这个跟前面介绍的如何使用nn.Block没什么区别。下面代码定义一个层将减掉均值 

In [1]:
from mxnet import nd 
from mxnet.gluon import nn

In [2]:
class CenteredLayer(nn.Block):
    def __init__(self,**kwargs):
        super(CenteredLayer,self).__init__(**kwargs)
    
    def forward(self,x): 
        return x-x.mean() 

我们可以马上实例化这个层用起来 

In [3]:
layer=CenteredLayer() 
layer(nd.array([1,2,3,4,5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

我们也可以用它来构造更复杂的神经网络 

In [4]:
net=nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(128))
    net.add(nn.Dense(10))
    net.add(CenteredLayer())

确认下输出的均值确实是0: 

In [5]:
net.initialize() 
y=net(nd.random.uniform(shape=(4,8))) 
print(y)
y.mean() 


[[-4.25069369e-02  2.76193079e-02  5.38392551e-02 -8.66032206e-05
  -4.78977449e-02 -1.96425803e-02  2.09131949e-02  6.18448434e-03
   3.31263221e-03 -2.44515762e-02]
 [-2.72416081e-02  5.39518101e-03  3.93499807e-02 -1.81009434e-02
  -2.04518754e-02 -2.84789596e-03  1.41671039e-02  2.61938758e-02
   2.30672173e-02 -3.95172043e-03]
 [-2.57457346e-02  3.63405645e-02  6.61725551e-02 -4.23212675e-03
  -4.59269471e-02 -3.13036293e-02  1.17697865e-02 -2.39959452e-03
   2.08159257e-02 -1.89681370e-02]
 [-1.96682289e-02  1.73189603e-02  1.62064843e-02 -6.78200973e-03
  -2.63251383e-02 -1.18263029e-02  1.67988148e-03  2.64359247e-02
  -9.39494930e-03 -7.03005167e-03]]
<NDArray 4x10 @cpu(0)>



[-6.635673e-10]
<NDArray 1 @cpu(0)>

当然大部分情况你可以看不到一个实实在在的0，而是一个很小的数。例如：5.645646356e-11。
这是因为 MXNet默认使用 32位float，会带来一定的浮点精度误差 

# 带模型参数的自定义层 
虽然Centeredlayer 可能会告诉实现自定义层大概是什么样子，但它缺少了最重要的一环，就是它没有可以学习的模型参数 

记得我们之前访问Dense 的权重的时候是通过dense.weight.data() ,这里的weight 是一个 Parameter 的类型。我们可以显示的构建这样一个参数。 

In [6]:
from mxnet import gluon
my_param=gluon.Parameter('exciting_parameter_yay',shape=(3,3)) 

In [7]:
my_param.initialize() 
(my_param.data(),my_param.grad())

(
 [[-0.02548236  0.05326662 -0.01200318]
  [ 0.05855297 -0.06101935 -0.0396449 ]
  [ 0.0269461   0.00912645  0.0093242 ]]
 <NDArray 3x3 @cpu(0)>,
 
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 <NDArray 3x3 @cpu(0)>)

通常自定义层的时候我们不会直接创建 Parameter ,而是通过BLOCK自带的一个ParameterDict 类型的成员变量
params,顾名思义，这是一个由字符串名字映射到Paramete的字典 。

In [8]:
pd=gluon.ParameterDict(prefix='block_') 
pd.get('exciting_parameter_yay',shape=(3,3)) 
pd 

block_ (
  Parameter block_exciting_parameter_yay (shape=(3, 3), dtype=<class 'numpy.float32'>)
)

现在我们看下如果实现一个跟Dense 一样功能的层，它概念跟前面的CenteredLayer 的主要区别
是我们在初始函数里通过Params创建了参数： 

In [9]:
class MyDense(nn.Block):
    def __init__(self,units,in_units,**kwargs):
        super(MyDense,self).__init__(**kwargs)
        with self.name_scope():
            self.weight=self.params.get(
            'weight',shape=(in_units,units))
            self.bias=self.params.get('bias',shape=(units,)) 
    def forward(self,x):
        linear=nd.dot(x,self.weight.data())+self.bias.data()
        return nd.relu(linear) 

我们创建实例化一个对象来看下它的参数，这里我们特意加了前缀prefix,这是nn.Block初始化函数自带的参数。 

In [10]:
dense=MyDense(5,in_units=10,prefix='o_my_dense_') 
dense.params 

o_my_dense_ (
  Parameter o_my_dense_weight (shape=(10, 5), dtype=<class 'numpy.float32'>)
  Parameter o_my_dense_bias (shape=(5,), dtype=<class 'numpy.float32'>)
)

它的使用跟前面没有什么不一致：

In [11]:
dense.initialize() 
dense(nd.random.uniform(shape=(2,10)))


[[0.         0.         0.0301928  0.09594411 0.13613266]
 [0.         0.         0.00460232 0.10275271 0.15692513]]
<NDArray 2x5 @cpu(0)>

我们构造的层跟Gluon 提供的层用起来没太多区别： 

In [12]:
net=nn.Sequential() 
with net.name_scope():
    net.add(MyDense(32,in_units=64))
    net.add(MyDense(2,in_units=32))
net.initialize() 
net(nd.random.uniform(shape=(2,64)))


[[0.         0.06250843]
 [0.00077506 0.08170694]]
<NDArray 2x2 @cpu(0)>

In [14]:
! nvidia-smi

Tue Dec 29 22:07:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 27%   25C    P8    19W / 250W |   1401MiB / 11016MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------